In [1]:
import dash
from dash import html, dcc
import plotly.express as px
from dash import dash_table
import pandas as pd
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
from dash_bootstrap_templates import load_figure_template

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [3]:
from io import BytesIO
import sys
import os

In [4]:
directory_path = os.path.abspath('../')
if directory_path not in sys.path:
    sys.path.append(directory_path)

import api_requests

In [5]:
pd.DataFrame.iteritems = pd.DataFrame.items

In [6]:
df_subjects = pd.read_csv(BytesIO(api_requests.fetch_data('subject?or=(dataset.eq.2020_dataset,dataset.eq.2021_dataset)').encode()))
df_specimen = pd.read_csv(BytesIO(api_requests.fetch_data('specimen').encode())) # includes 2022 data

new = pd.merge(df_subjects, df_specimen, on= 'subject_id')

In [7]:
new

,subject_id,infancy_vac,biological_sex,ethnicity,race,year_of_birth,date_of_boost,dataset,specimen_id,actual_day_relative_to_boost,planned_day_relative_to_boost,specimen_type,visit
0,1,wP,Female,Not Hispanic or Latino,White,1986-01-01,2016-09-12,2020_dataset,1,-3,0,Blood,1
1,1,wP,Female,Not Hispanic or Latino,White,1986-01-01,2016-09-12,2020_dataset,2,1,1,Blood,2
2,1,wP,Female,Not Hispanic or Latino,White,1986-01-01,2016-09-12,2020_dataset,3,3,3,Blood,3
3,1,wP,Female,Not Hispanic or Latino,White,1986-01-01,2016-09-12,2020_dataset,4,7,7,Blood,4
4,1,wP,Female,Not Hispanic or Latino,White,1986-01-01,2016-09-12,2020_dataset,5,11,14,Blood,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,96,aP,Male,Hispanic or Latino,Unknown or Not Reported,2000-01-01,2019-06-24,2021_dataset,725,3,3,Blood,3
725,96,aP,Male,Hispanic or Latino,Unknown or Not Reported,2000-01-01,2019-06-24,2021_dataset,726,7,7,Blood,4
726,96,aP,Male,Hispanic or Latino,Unknown or Not Reported,2000-01-01,2019-06-24,2021_dataset,727,14,14,Blood,5
727,96,aP,Male,Hispanic or Latino,Unknown or Not Reported,2000-01-01,2019-06-24,2021_dataset,728,30,30,Blood,6


In [8]:
cols_dd = ['1.1 Rank individuals by IgG titers against PT in plasma 14 days post booster',
           '1.2 Rank individuals by fold change in IgG titers against PT from day 0 to day 14 post booster',
           '2.1 Rank individuals by predicted monocyte frequency on day 1 post booster',
           '2.2 Rank individuals by fold change in predicted monocyte frequency from day 0 to day 1 post booster',
           '3.1 Rank individuals by predicted CCL3 gene expression on day 3 post booster',
           '3.2 Rank individuals by fold change in predicted CCL3 gene expression from day 0 to day 3 post booster']

drops = html.Div([html.Label('Select Task:'),
                  dcc.Dropdown(
                      id='demo-dropdown',
                      options=[{'label': k, 'value': k} for k in cols_dd],
                      value=cols_dd[0],
                      optionHeight=60
                  )])

#cols_dd2 = ['MSE', 'MAE', 'R-squared']

drops2 = html.Div([html.Label('Select Evaluation Metric:'),
                  dcc.Dropdown(
                      id='metric-dropdown',
                      options=['MSE', 'MAE', 'R-squared'],
                      value='MSE',
                  )])


In [9]:
task11fs = pd.read_csv('Downloads1/task11_feature_selected.csv')
task12fs = pd.read_csv('Downloads1/task12_feature_selected.csv')
task21fs = pd.read_csv('Downloads1/task21_feature_selected.csv')
task22fs = pd.read_csv('Downloads1/task22_feature_selected.csv')
task31fs = pd.read_csv('Downloads1/task31_feature_selected.csv')
task32fs = pd.read_csv('Downloads1/task32_feature_selected.csv')

df_train1 = pd.read_csv('df_train1.csv')
df_train2 = pd.read_csv('df_train2.csv')
df_d3 = pd.read_csv('df_d3.csv')

df = pd.read_csv('Prediction_value.csv').round(3)

In [ ]:
load_figure_template([ 'minty'])#"minty", "minty_dark",
app =  dash.Dash(
    __name__, 
    external_stylesheets=[
        dbc.themes.MINTY, 
        dbc.icons.BOOTSTRAP
    ],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1"},
        {'name': 'description', 'content': 'A dashboard displaying immune responses to pertussis vaccinations.'},
    ],
)

app.title = "CMI-PB Responses Pertussis Vaccination"


r2 = dbc.Card(dcc.Graph(id='r2', style={'padding': 8, 'flex': 1}))
imm = dbc.Card(dcc.Graph(id='imm', style={'padding': 8, 'flex': 1}))
eva = dbc.Card(dcc.Graph(id='eval', style={'padding': 8, 'flex': 1}))
res = dbc.Card(dcc.Graph(id='results', style={'padding': 8, 'flex': 1}))
age= dbc.Card(dcc.Graph(id='age', style={'padding': 4, 'flex': 1}))
sex= dbc.Card(dcc.Graph(id='sex', style={'padding': 4, 'flex': 1}))
race= dbc.Card(dcc.Graph(id='race', style={'padding': 4, 'flex': 1}))


app.layout = html.Div([
    dbc.NavbarSimple(
        children=[
            dbc.NavItem(dbc.NavLink("Data", href="https://www.cmi-pb.org/data")),
            dbc.NavItem(
                dbc.NavLink(
                    html.Span(
                        [html.I(className="bi bi-github me-1")," GitHub"]),
                        href="https://github.com/brianrqian/DSE260A-Immune-Response",
                )
            ),
        ],
        brand="CMI-PB Responses Pertussis Vaccination",
        brand_href="https://www.cmi-pb.org/", 
        color="dark",  
        dark=True, 
        className='mb-4 sticky-top',
        fluid=True
    ),
    dbc.Container(
        dbc.Stack([
            dbc.Row([
                dbc.Col(dcc.Graph(id='age', style={'height': '500px', 'width': '100%'}), width=3),
                dbc.Col(dcc.Graph(id='race', style={'height': '500px', 'width': '100%'}), width=3),
                dbc.Col(dcc.Graph(id='sex', style={'height': '500px', 'width': '100%'}), width=3),
                dbc.Col(dcc.Graph(id='bubble-chart', style={'height': '500px', 'width': '100%'}), width=3)
            ]),
            dbc.Row([dbc.Col(drops), dbc.Col(drops2)]),
            dbc.Row([
                dbc.Col(dcc.Graph(id='imm', style={'height': '500px', 'width': '100%'}), width=6),
                dbc.Col(dcc.Graph(id='r2', style={'height': '500px', 'width': '100%'}), width=6)
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(id='parallel-coords-plot', style={'height': '500px', 'width': '100%'}), width=12)
            ]),
            dbc.Row(
                dbc.Col(
                    html.Div([
                        dbc.Button("Download CSV", color="dark", id="btn_csv"),
                        dcc.Download(id="download-dataframe-csv")
                    ])
                )
            ),
            dbc.Row([
                dbc.Col([
                    dash_table.DataTable(
                    id='result-table',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict('records'),
                    page_size=len(df),  
                    style_cell={  
                        'fontSize': '12px',
                        'font-family': 'system-ui',
                        'maxWidth': '150px',
                        'whiteSpace': 'normal',
                        'height': 'auto',
                        'textAlign': 'left'
                    },
                    style_header={
                        'backgroundColor': '#343a40',
                        'color': 'white',
                        'fontWeight': 'bold'
                    },
                    style_table={ 
                        'width': '100%',  
                        'minWidth': '100%'
                    },
                    style_data={
                        'color': 'black'
                    },
                    )
                ])
            ])
        ], 
        gap=4
                 
        ),
        fluid=True,
    )
])


@app.callback(
    dash.dependencies.Output('age', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def generate_age_distribution(df):
    df_train = pd.read_csv('df_train1.csv')
    fig = px.histogram(df_train, x='Age', nbins=20, title="Age Distribution").update_layout(template="minty", bargap=0.2) 
    fig.update_layout(title_x=0.5)
    return fig


@app.callback(
    dash.dependencies.Output('sex', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def generate_sex_distribution(df):
    df_train1 = pd.read_csv('df_train1.csv')

    df = df_train1
    sex_distribution = df['biological_sex_Male'].map({1.0: 'Male', 0.0: 'Female'})
    fig = px.pie(names=sex_distribution, title="Sex Distribution").update_layout(template="minty",)
    fig.update_layout(title_x=0.5)
    return fig

@app.callback(
    dash.dependencies.Output('race', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def generate_race_distribution(df):
    df = new
    fig = px.histogram(df, x='race', title="Race Distribution").update_layout(template="minty",)
    fig.update_layout(title_x=0.5)
    return fig


@app.callback(
    dash.dependencies.Output('imm', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def generate_immunogenicity_plot(value):
    df_train1 = pd.read_csv('df_train1.csv')#.round(3)
    df_train2 = pd.read_csv('df_train2.csv')#.round(3)
    df_d3 = pd.read_csv('df_d3.csv')#.round(3)
    try:
        if value == '1.1 Rank individuals by IgG titers against PT in plasma 14 days post booster':
            df_train = df_train1
        elif value == '1.2 Rank individuals by fold change in IgG titers against PT from day 0 to day 14 post booster':
            df_train = df_train1
        elif value == '2.1 Rank individuals by predicted monocyte frequency on day 1 post booster':
            df_train = df_train2
        elif value == '2.2 Rank individuals by fold change in predicted monocyte frequency from day 0 to day 1 post booster':
            df_train = df_train2
        elif value == '3.1 Rank individuals by predicted CCL3 gene expression on day 3 post booster':
            df_train = df_d3
        elif value == '3.2 Rank individuals by fold change in predicted CCL3 gene expression from day 0 to day 3 post booster':
            df_train = df_d3
        else:
            return {}

        df_selected = df_train.iloc[:, :9]
        correlation_matrix = df_selected.corr().round(3)

        fig = px.imshow(correlation_matrix,
                        text_auto=True, 
                        labels=dict(color='Correlation'),
                        x=correlation_matrix.columns,
                        y=correlation_matrix.columns,
                        title="Feature Correlations Heatmap",
                        #color_continuous_scale=[[0, "#caf3ff"], [1, "#2c82ff"]],
                        aspect='auto')

        fig.update_layout(
            #autosize=False,
            #width=650, 
            #height=400,
            #font=dict(size=9),
        )
        fig.update_layout(template="minty",title_x=0.5)

        return fig

    except Exception as e:
        print(f"Error: {e}")
        return {}



@app.callback(
    dash.dependencies.Output('r2', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value'), 
    dash.dependencies.Input('metric-dropdown', 'value')])
def generate_model_evaluation_chart(value, met):
    task11ev = pd.read_csv('Downloads1/task11_model_evaluation.csv')
    task12ev = pd.read_csv('Downloads1/task12_model_evaluation.csv')
    task21ev = pd.read_csv('Downloads1/task21_model_evaluation.csv')
    task22ev = pd.read_csv('Downloads1/task22_model_evaluation.csv')
    task31ev = pd.read_csv('Downloads1/task31_model_evaluation.csv')
    task32ev = pd.read_csv('Downloads1/task32_model_evaluation.csv')

    datasets = {
        '1.1 Rank individuals by IgG titers against PT in plasma 14 days post booster': task11ev,
        '1.2 Rank individuals by fold change in IgG titers against PT from day 0 to day 14 post booster': task12ev,
        '2.1 Rank individuals by predicted monocyte frequency on day 1 post booster': task21ev,
        '2.2 Rank individuals by fold change in predicted monocyte frequency from day 0 to day 1 post booster': task22ev,
        '3.1 Rank individuals by predicted CCL3 gene expression on day 3 post booster': task31ev,
        '3.2 Rank individuals by fold change in predicted CCL3 gene expression from day 0 to day 3 post booster': task32ev
    }

    data = datasets[value]
    if met == 'MAE':
        ys ='MAE'
    elif met == 'MSE':
        ys = 'MSE'
    else:
        ys = 'R-squared'
        

    fig = px.bar(data, x='Model', y=ys, labels={'x': 'Model', 'y': ys}, title=f"{met} Model Evaluation for {value[:3]}").update_yaxes(rangemode='tozero')
    fig.update_layout(template="minty",)
    fig.update_layout(title_x=0.5)
    
    #fig.update_yaxes(rangemode='tozero')
    return fig


@app.callback(
    dash.dependencies.Output('bubble-chart', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def generate_bubble_chart(value):
    df_train1 = pd.read_csv('df_train1.csv').round(3)
    df_train2 = pd.read_csv('df_train2.csv').round(3)
    df_d3 = pd.read_csv('df_d3.csv').round(3)
    if value:
        df = df_d3
        selected_data = df.iloc[0, 9:]  

        bubble_df = pd.DataFrame({
            'Feature Index': range(len(selected_data)),
            'Value': selected_data.values,
            'Feature': selected_data.index 
        })

        bubble_df['Size'] = bubble_df['Value'] / bubble_df['Value'].max() * 40

        fig = px.scatter(
            bubble_df, 
            x='Value', 
            y='Feature Index', 
            size='Size', 
            color='Value', 
            #color_continuous_scale=[[0, "#caf3ff"], [1, "#2c82ff"]],#px.colors.sequential.Viridis, 
            title='Bubble Chart for Features',
            hover_data=['Feature'],
            size_max=30 
        )

        max_value = bubble_df['Value'].max()
        fig.update_layout(template="minty",title_x=0.5)
        fig.update_xaxes(range=[0, max_value * 1.1]) 

        fig.update_yaxes(showticklabels=False)

        return fig
    return {} 



@app.callback(
    dash.dependencies.Output('parallel-coords-plot', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')]) 
def update_parallel_coords(value):
    task11fs = pd.read_csv('Downloads1/task11_feature_selected.csv')
    task12fs = pd.read_csv('Downloads1/task12_feature_selected.csv')
    task21fs = pd.read_csv('Downloads1/task21_feature_selected.csv')
    task22fs = pd.read_csv('Downloads1/task22_feature_selected.csv')
    task31fs = pd.read_csv('Downloads1/task31_feature_selected.csv')
    task32fs = pd.read_csv('Downloads1/task32_feature_selected.csv')

    task_names = ['task11', 'task12', 'task21', 'task22', 'task31', 'task32']

    common_features = set(task11fs['Selected Features']).union(task12fs['Selected Features']).union(
        task21fs['Selected Features']).union(task22fs['Selected Features']).union(
        task31fs['Selected Features']).union(task32fs['Selected Features'])
    all_data = pd.DataFrame(index=list(common_features))

    datasets = [task11fs, task12fs, task21fs, task22fs, task31fs, task32fs]
    for i, dataset in enumerate(datasets):
        dataset.set_index('Selected Features', inplace=True)
        all_data = all_data.join(dataset[['Score']].rename(columns={'Score': f'Score_{task_names[i]}'}), how='left').fillna(0)

    all_data.reset_index(inplace=True)
    all_data.rename(columns={'index': 'Features'}, inplace=True)
    if value == '1.1 Rank individuals by IgG titers against PT in plasma 14 days post booster':
        colors = 'Score_task11'
    elif value == '1.2 Rank individuals by fold change in IgG titers against PT from day 0 to day 14 post booster':
        colors = 'Score_task12'
    elif value == '2.1 Rank individuals by predicted monocyte frequency on day 1 post booster':
        colors = 'Score_task21'
    elif value == '2.2 Rank individuals by fold change in predicted monocyte frequency from day 0 to day 1 post booster':
        colors = 'Score_task22'
    elif value == '3.1 Rank individuals by predicted CCL3 gene expression on day 3 post booster':
        colors = 'Score_task31'
    elif value == '3.2 Rank individuals by fold change in predicted CCL3 gene expression from day 0 to day 3 post booster':
        colors = 'Score_task32'

    fig = px.parallel_coordinates(all_data, color= colors,
                                  dimensions=[col for col in all_data.columns if col.startswith('Score_')],
                                  labels={col: col for col in all_data.columns if col.startswith('Score_')},
                                  color_continuous_scale=px.colors.diverging.Tealrose,
                                  color_continuous_midpoint=0)
    fig.update_layout(template="minty",title_x=0.5)
    return fig

@app.callback(
    dash.dependencies.Output("download-dataframe-csv", "data"),
    dash.dependencies.Input("btn_csv", "n_clicks"),
    prevent_initial_call=True
)
def generate_csv(n_clicks):
    return dcc.send_data_frame(df.to_csv, "Files/Prediction_value.csv")


if __name__ == '__main__':
    app.server.run(port=8003, host='127.0.0.1')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8003
Press CTRL+C to quit
127.0.0.1 - - [01/Jun/2024 14:48:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:48:50] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_16_1m1714525179.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:48:50] "GET /_dash-component-suites/dash/deps/react@16.v2_16_1m1714525179.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:48:50] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_16_1m1714525179.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:48:50] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_16_1m1714525179.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:48:50] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_6_0m1714525223.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:48:50] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_16_1m1714525179.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 